### Assignment1

In [27]:
import nltk
import pandas as pd
from nltk.corpus import stopwords
from unidecode import unidecode
#nltk.download('stopwords') 
stopwords.words('english')
df = pd.read_csv("https://bit.ly/nlp-reuters")
corpus = df.content
from nltk.tokenize import word_tokenize
#bit.ly/nlp-reuters
#tokens = word_tokenize("Tw Taiwan gives 400,000 masks to U.S. Taiwan under cooperation arrangement. #TaiwanCanHelp:)")
data=[word_tokenize(unidecode(docu))for docu in corpus ]
from nltk import pos_tag

In [28]:
from nltk import ngrams
t=[]
for i in range(len(data)):
    ls=list(ngrams(data[i],2))
    pos = pos_tag(data[i])
    for i in range(len(ls)):
        if pos[i][1]=='NNP' and pos[i+1][1]=='NNP':
            t.append(ls[i])

In [29]:
from collections import Counter
count=Counter(t)
count.most_common(5)

[(('U.', 'S.'), 17601),
 (('Donald', 'Trump'), 3217),
 (('New', 'York'), 2443),
 (('Islamic', 'State'), 1925),
 (('President', 'Donald'), 1922)]

### Assignment2

In [30]:
import spacy
#!python -m spacy download en_core_web_lg
df = pd.read_csv("http://bit.ly/nlp-buzzfeed")
contents = df.content
titles = df.title
import en_core_web_sm
nlp = en_core_web_sm.load()


In [31]:
idx = [i for i, x in enumerate(df['content'].isna()) if x]
print(idx)

[60, 233, 488]


In [32]:
vocabulary = Counter()
tokens=[]
for i in range(len(contents)):
    if i in idx:
        continue
    docu = nlp(contents[i])
    tok=[]
    for token in docu:
        if token.is_stop !=True:
            tok.append(token.lemma_+'_'+token.pos_)
    tokens.append(tok)
    vocabulary.update(tok)

In [18]:
#print(vocabulary.most_common(1000))

In [33]:
import math
def tfidf_vectors(corpus, vocab):
    def tfidf_vec(doc):
        doc_freqs = Counter(doc)
        return [doc_freqs[token] * math.log(N / df[token]) for token, _ in vocab]
    df = Counter()
    token_set = set([token for token, _ in vocab])
    for document in corpus:
        df.update(list(set(document) & token_set))
        #print(df.most_common(10))
    N = len(corpus)
    # use df to compute the df-idf vector for each document
    return [tfidf_vec(doc) for doc in corpus]
tf_score = tfidf_vectors(tokens,vocabulary.most_common(512))

In [34]:
def cosine_similarity(vec_a, vec_b):
    assert len(vec_a) == len(vec_b)
    a_b = sum(i[0] * i[1] for i in zip(vec_a, vec_b))
    a_2 = sum([i*i for i in vec_a])
    b_2 = sum([i*i for i in vec_b])
    if a_2 == 0 or b_2 == 0:
        return float('nan')
    return a_b/(math.sqrt(a_2) * math.sqrt(b_2))


In [35]:
def k_similar(seed_id, vectors, k=5):
    similarities = [cosine_similarity(vectors[seed_id], vectors[id]) for id in range(len(vectors))]
    top_indices = sorted(range(len(similarities)), key=lambda i: similarities[i])[-k:] # https://stackoverflow.com/questions/13070461/get-indices-of-the-top-n-values-of-a-list
    return [(id, similarities[id]) for id in reversed(top_indices)]

In [36]:
seed_id = 719
i=k_similar(seed_id , tf_score)
print("* {}".format(titles[seed_id]))
for id, similarity in i:
    print(" {:4} > {} ({:.4f})".format(id, titles[id], similarity))
    

* This Woman Reimagined Michelangelo’s "The Creation Of Adam" With Black Women And It’s Beautiful
  719 > This Woman Reimagined Michelangelo’s "The Creation Of Adam" With Black Women And It’s Beautiful (1.0000)
  640 > Scientists Are Arguing About Whether The March For Science Will Be Too Political (0.6273)
  625 > Trump Says Obamacare Will “Explode” After Replacement Fails In Congress (0.6133)
  368 > Donald Trump Admits He Supported “Surgical” Intervention In Libya (0.5895)
  820 > Latinos Are Expected To Vote In Droves But Major Liberal Efforts To Register Them Aren’t Off The Ground (0.5782)
